# Setup Environment and Read Data

In [1]:
import torch
import numpy as np
import pandas as pd
import pickle
import copy
from tqdm import trange,tqdm
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, classification_report, confusion_matrix

## Download the dataset

In [ ]:
!wget https://raw.githubusercontent.com/khundman/telemanom/master/labeled_anomalies.csv

--2025-04-21 14:04:03--  https://raw.githubusercontent.com/khundman/telemanom/master/labeled_anomalies.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3956 (3.9K) [text/plain]
Saving to: ‘labeled_anomalies.csv’

labeled_anomalies.c 100%[===================>]   3.86K  --.-KB/s    in 0s      

2025-04-21 14:04:04 (32.3 MB/s) - ‘labeled_anomalies.csv’ saved [3956/3956]



In [ ]:
%env DRIVE_PATH=/content/drive/MyDrive/Colab Notebooks/ELTE/DSLAB/

env: DRIVE_PATH=/content/drive/MyDrive/Colab Notebooks/ELTE/DSLAB/


In [ ]:
!mkdir "/root/.config/kaggle"

In [ ]:
!cp "$DRIVE_PATH/kaggle.json" "/root/.config/kaggle"
!chmod 600 "/root/.config/kaggle/kaggle.json"

In [ ]:
!cd "$DRIVE_PATH" && kaggle datasets download -d patrickfleith/nasa-anomaly-detection-dataset-smap-msl && mv nasa-anomaly-detection-dataset-smap-msl.zip data.zip && unzip -o data.zip && rm data.zip && mv data/data tmp && rm -r data && mv tmp data

Dataset URL: https://www.kaggle.com/datasets/patrickfleith/nasa-anomaly-detection-dataset-smap-msl
License(s): copyright-authors
Archive:  data.zip
  inflating: data/data/2018-05-19_15.00.10/models/A-1.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-2.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-3.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-4.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-5.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-6.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-7.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-8.h5  
  inflating: data/data/2018-05-19_15.00.10/models/A-9.h5  
  inflating: data/data/2018-05-19_15.00.10/models/B-1.h5  
  inflating: data/data/2018-05-19_15.00.10/models/C-1.h5  
  inflating: data/data/2018-05-19_15.00.10/models/C-2.h5  
  inflating: data/data/2018-05-19_15.00.10/models/D-1.h5  
  inflating: data/data/2018-05-19_15.00.10/models/D-11.h5  
  inflating: data/data/20

## Setup the dataset

In [2]:
DRIVE = "drive/MyDrive/Colab Notebooks/ELTE/DSLAB/ServerMachineDataset/"
MACHINE = "machine-1-1.txt"
TRAIN_DATASET = DRIVE + "train/" + MACHINE
TEST_DATASET = DRIVE + "test/" + MACHINE
TEST_LABEL_DATASET = DRIVE + "test_label/" + MACHINE

metric = pd.read_csv(TRAIN_DATASET, header=None)
metric_test = pd.read_csv(TEST_DATASET, header=None)
true_anomalies = pd.read_csv(TEST_LABEL_DATASET, header=None)[0].to_numpy()

In [3]:
metric

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.032258,0.039195,0.027871,0.024390,0.0,0.915385,0.343691,0.0,0.020011,0.000122,...,0.0,0.004298,0.029993,0.022131,0.000000,0.000045,0.034677,0.034747,0.0,0.0
1,0.043011,0.048729,0.033445,0.025552,0.0,0.915385,0.344633,0.0,0.019160,0.001722,...,0.0,0.004298,0.030041,0.028821,0.000000,0.000045,0.035763,0.035833,0.0,0.0
2,0.043011,0.034958,0.032330,0.025552,0.0,0.915385,0.344633,0.0,0.020011,0.000122,...,0.0,0.004298,0.026248,0.021101,0.000000,0.000045,0.033012,0.033082,0.0,0.0
3,0.032258,0.028602,0.030100,0.024390,0.0,0.912821,0.342750,0.0,0.021289,0.000000,...,0.0,0.004298,0.030169,0.025733,0.000000,0.000022,0.035112,0.035182,0.0,0.0
4,0.032258,0.019068,0.026756,0.023229,0.0,0.912821,0.342750,0.0,0.018734,0.000000,...,0.0,0.004298,0.027240,0.022645,0.000000,0.000034,0.033447,0.033517,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.075269,0.046610,0.071349,0.076655,0.0,0.928205,0.269303,0.0,0.031649,0.000244,...,0.0,0.008596,0.068980,0.049408,0.000386,0.000034,0.064504,0.064572,0.0,0.0
28475,0.086022,0.070975,0.075808,0.077816,0.0,0.930769,0.269303,0.0,0.029946,0.000244,...,0.0,0.008596,0.073029,0.055584,0.000386,0.000034,0.067690,0.067757,0.0,0.0
28476,0.086022,0.065678,0.073579,0.076655,0.0,0.935897,0.270245,0.0,0.030372,0.000244,...,0.0,0.008596,0.070516,0.048893,0.000386,0.000034,0.064866,0.064934,0.0,0.0
28477,0.086022,0.056144,0.068004,0.074332,0.0,0.933333,0.271186,0.0,0.032643,0.000244,...,0.0,0.008596,0.070308,0.055069,0.000386,0.000045,0.067111,0.067178,0.0,0.0


# Preprocess the Dataset

In [3]:
# Scale the values of the input metrics
scaler = MinMaxScaler()
metric_scaled = scaler.fit_transform(metric)
metric_tensor = torch.tensor(metric_scaled, dtype=torch.float32)
metric_scaled = pd.DataFrame(metric_scaled, index=metric.index, columns=metric.columns)
metric_scaled

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.065217,0.092965,0.100001,0.097221,0.0,0.915385,0.953003,0.0,0.169468,0.000869,...,0.0,0.150002,0.088565,0.052083,0.000000,0.035405,0.085685,0.085926,0.0,0.0
1,0.086957,0.115578,0.120001,0.101853,0.0,0.915385,0.955615,0.0,0.162261,0.012262,...,0.0,0.150002,0.088743,0.074651,0.000000,0.035405,0.089275,0.089517,0.0,0.0
2,0.086957,0.082915,0.116000,0.101853,0.0,0.915385,0.955615,0.0,0.169468,0.000869,...,0.0,0.150002,0.074656,0.048609,0.000000,0.035405,0.080180,0.080421,0.0,0.0
3,0.065217,0.067840,0.107999,0.097221,0.0,0.912821,0.950394,0.0,0.180291,0.000000,...,0.0,0.150002,0.089218,0.064234,0.000000,0.017309,0.087123,0.087364,0.0,0.0
4,0.065217,0.045227,0.096000,0.092593,0.0,0.912821,0.950394,0.0,0.158654,0.000000,...,0.0,0.150002,0.078340,0.053817,0.000000,0.026751,0.081618,0.081859,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.152174,0.110552,0.256000,0.305555,0.0,0.928205,0.746736,0.0,0.268028,0.001737,...,0.0,0.300003,0.233357,0.144096,0.166523,0.026751,0.184297,0.184538,0.0,0.0
28475,0.173914,0.168343,0.271999,0.310183,0.0,0.930769,0.746736,0.0,0.253606,0.001737,...,0.0,0.300003,0.248395,0.164929,0.166523,0.026751,0.194830,0.195069,0.0,0.0
28476,0.173914,0.155779,0.264001,0.305555,0.0,0.935897,0.749348,0.0,0.257213,0.001737,...,0.0,0.300003,0.239062,0.142359,0.166523,0.026751,0.185493,0.185735,0.0,0.0
28477,0.173914,0.133166,0.243998,0.296296,0.0,0.933333,0.751958,0.0,0.276446,0.001737,...,0.0,0.300003,0.238289,0.163192,0.166523,0.035405,0.192916,0.193155,0.0,0.0


### Scaled

In [17]:
# create train and test dataloaders
metric.interpolate(inplace=True)
metric.bfill(inplace=True)
metric_tensor = metric.values

metric_test.interpolate(inplace=True)
metric_test.bfill(inplace=True)
metric_test_tensor = metric_test.values

sequence_length = 30
sequences = []
for i in range(metric_tensor.shape[0] - sequence_length + 1):
  sequences.append(metric_tensor[i:i + sequence_length])


train_data, val_data = train_test_split(sequences, test_size=0.3, random_state=42) # 70% train, 30% temp

test_sequences = []
for i in range(metric_test_tensor.shape[0] - sequence_length + 1):
  test_sequences.append(metric_test_tensor[i:i + sequence_length])


batch_size = 32
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_sequences, batch_size=batch_size, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
sequences[0].shape

(30, 38)

# Define the Network

## LSTM

In [19]:
class LSTMEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers=1):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)  # h_n: (num_layers, batch, hidden_dim)
        h = h_n[-1]  # take the output of the last layer
        return self.fc_mean(h), self.fc_logvar(h)


class LSTMDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, sequence_length, num_layers=1):
        super(LSTMDecoder, self).__init__()
        self.sequence_length = sequence_length
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, z):
        # Repeat z for each timestep
        hidden = self.latent_to_hidden(z).unsqueeze(1).repeat(1, self.sequence_length, 1)
        out, _ = self.lstm(hidden)
        return self.output_layer(out)


class LSTMVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, sequence_length, num_layers=1, device='cpu'):
        super(LSTMVAE, self).__init__()
        self.encoder = LSTMEncoder(input_dim, hidden_dim, latent_dim, num_layers).to(device)
        self.decoder = LSTMDecoder(latent_dim, hidden_dim, input_dim, sequence_length, num_layers).to(device)

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x):
        mean, logvar = self.encoder(x)
        z = self.reparameterize(mean, logvar)
        x_recon = self.decoder(z)
        return x_recon, mean, logvar

In [20]:
input_dim = 38
hidden_dim = 128
latent_dim = 32
num_layers = 1

model = LSTMVAE(input_dim=input_dim,
                hidden_dim=hidden_dim,
                latent_dim=latent_dim,
                sequence_length=sequence_length,
                num_layers=num_layers,
                device=device).to(device)
optimizer = Adam(model.parameters(), lr=1e-3)

## Support functions

In [21]:
def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.mse_loss(x_hat, x, reduction='sum')
    KLD = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss + KLD

In [ ]:
def save_model(model):
    model_state = {
        'input_dim':input_dim,
        'latent_dim':latent_dim,
        'hidden_dim':hidden_dim,
        'state_dict':model.state_dict()
    }
    torch.save(model_state,'vae.pth')

# Train

## LSTM

In [23]:
torch.cuda.empty_cache()

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)

# SPO optimizer - optuna
# bayesian hyperparameter tuning
# grid search - slow for DL

def train_model(model, train_loader, val_loader, optimizer, loss_fn, scheduler, num_epochs=10, device='cpu'):
    torch.cuda.empty_cache()
    train_losses = []
    val_losses = []

    early_stop_tolerant_count = 0
    early_stop_tolerant = 10
    best_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        train_loss = 0.0
        model.train()
        for batch in train_loader:
            batch = torch.tensor(batch, dtype=torch.float32).to(device)

            optimizer.zero_grad()

            recon_batch, mean, logvar = model(batch)
            loss = loss_fn(recon_batch, batch, mean, logvar)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        valid_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                batch = torch.tensor(batch, dtype=torch.float32).to(device)
                recon_batch, mean, logvar = model(batch)
                loss = loss_fn(recon_batch, batch, mean, logvar)
                valid_loss += loss.item()

        valid_loss /= len(val_loader)
        val_losses.append(valid_loss)

        scheduler.step(valid_loss)

        if valid_loss < best_loss:
            best_loss = valid_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop_tolerant_count = 0
        else:
            early_stop_tolerant_count += 1

        print(f"Epoch {epoch+1:04d}: train loss {train_loss:.4f}, valid loss {valid_loss:.4f}")

        if early_stop_tolerant_count >= early_stop_tolerant:
            print("Early stopping triggered.")
            break

    model.load_state_dict(best_model_wts)
    print("Finished Training.")
    return train_losses, val_losses

train_losses, val_losses = train_model(model, train_loader, val_loader, optimizer, loss_function, scheduler, num_epochs=100, device=device)

/tmp/ipython-input-2253947529.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)
/tmp/ipython-input-2253947529.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)


Epoch 0001: train loss 114.2897, valid loss 66.7853
Epoch 0002: train loss 65.0945, valid loss 66.8104
Epoch 0003: train loss 63.9090, valid loss 63.8125
Epoch 0004: train loss 63.7886, valid loss 64.0277
Epoch 0005: train loss 63.1664, valid loss 63.6343
Epoch 0006: train loss 62.7351, valid loss 63.6573
Epoch 0007: train loss 62.7799, valid loss 64.1735
Epoch 0008: train loss 62.6212, valid loss 62.7316
Epoch 0009: train loss 62.2323, valid loss 62.8784
Epoch 0010: train loss 62.4976, valid loss 63.0214
Epoch 0011: train loss 61.9804, valid loss 62.3462
Epoch 0012: train loss 62.2056, valid loss 62.3229
Epoch 0013: train loss 62.3023, valid loss 62.8909
Epoch 0014: train loss 61.7821, valid loss 62.2296
Epoch 0015: train loss 62.0300, valid loss 62.8738
Epoch 0016: train loss 61.8883, valid loss 62.1202
Epoch 0017: train loss 61.7317, valid loss 62.9656
Epoch 0018: train loss 61.7323, valid loss 62.2831
Epoch 0019: train loss 61.8926, valid loss 62.0265
Epoch 0020: train loss 61.5962

# Evaluate

In [24]:
def evaluate_lstm(model, test_loader, device, percentile_threshold=90):
    model.eval()
    anomaly_scores = []

    with torch.no_grad():
        for batch in test_loader:
            batch = torch.tensor(batch, dtype=torch.float32).to(device)

            batch_scores = []
            for i in range(batch.shape[0]): #Iterate through each sequence in the batch
                sequence = batch[i, :, :].unsqueeze(0)  # Select a single sequence
                recon_batch, mean, logvar = model(sequence)
                loss = loss_function(recon_batch, sequence, mean, logvar)
                batch_scores.append(loss.item())
            anomaly_scores.extend(batch_scores)  # Append scores for all sequences in the batch


    # Calculate the threshold based on the specified percentile
    threshold = np.percentile(anomaly_scores, percentile_threshold)

    # Identify anomaly indices
    anomaly_indices = [i for i, score in enumerate(anomaly_scores) if score > threshold]
    return anomaly_indices
anomalies = evaluate_lstm(model, test_loader, device, 90)

/tmp/ipython-input-190499672.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)


In [25]:
def calculate_f1_score(anomaly_indices, true_anomalies):
    # Create a binary array representing predicted anomalies
    predicted_anomalies = np.zeros_like(true_anomalies)
    for index in anomaly_indices:
        if index < len(predicted_anomalies):  # Check index bounds
          predicted_anomalies[index] = 1

    # Calculate the F1 score
    f1 = f1_score(true_anomalies, predicted_anomalies)
    return f1, predicted_anomalies

# Example usage (assuming 'anomalies' and 'true_anomalies' are defined)
f1, predicted_anomalies = calculate_f1_score(anomalies, true_anomalies)
print(f"F1 Score: {f1}")

F1 Score: 0.6185231991334176


In [26]:
print(classification_report(true_anomalies, predicted_anomalies))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     25785
           1       0.60      0.64      0.62      2694

    accuracy                           0.93     28479
   macro avg       0.78      0.80      0.79     28479
weighted avg       0.93      0.93      0.93     28479



In [27]:
print(confusion_matrix(true_anomalies, predicted_anomalies))

[[24653  1132]
 [  981  1713]]
